### Bibliotecas

In [3]:
# Matematicos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados e arquivos
import re
import requests
from pathlib import Path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# GetDate
from datetime import datetime #,  date

### Importacao de listagem

In [6]:
#Reimportando arquivo - caso execucao tenha sido pausada
path_to_file = '..//ListagemExp/ListagemExport2012-2015_checkpoint_14-11-2022_20-15-39.csv'
listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

### Requisicao a Camara

In [14]:
# Preenchimento do campo Data_Projeto, Tramitacao_Encerrada e Aprovado
count = 1
failed_req= 0
failed = False
max_failed_req = 2
req_count = 0
max_req = 3000
export = False

data_projeto = ''
notfound_count_dt = 0

tramitacao_encerrada = ''
notfound_count_tram = 0

aprovado = ''
notfound_count_aprv = 0

while count > 0:
    for i in listagem.index:
        if (listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '') and listagem['Id_Projeto_Camara'][i] != '': 
            with requests.Session() as session:
                data_projeto = ''
                tramitacao_encerrada = ''
                aprovado = ''

                failed = False
                # Variaveis de requisicao
                id_proj = listagem['Id_Projeto_Camara'][i]

                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"num": id_proj}
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
#                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', '').replace('\n', '').replace('\r', '').replace('\t', ''))
                    req_count+=1

                    #Identificacao da data de registro do projeto
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                    except:
                        data_projeto = 'Data nao encontrada'
                        notfound_count_dt += 1


                    #Identificacao do encerramento da tramitacao do projeto
                    #if 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jur' in listagem['Situacao'][i]:    
                    if 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jur' in listagem['Situacao'][i] or 'Retirad' in listagem['Situacao'][i] or 'Rejeitad' in listagem['Situacao'][i] or 'Vetado Totalmente' in listagem['Situacao'][i]:    
                            tramitacao_encerrada = 'SIM'
                    else:
                        try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                        
                            if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                                tramitacao_encerrada = 'SIM'
                            else:
                                tramitacao_encerrada = 'NAO'
                        except:
                            tramitacao_encerrada = 'Informacao nao encontrada'
                            notfound_count_tram+=1


                    #Identificacao da aprovacao do projeto
                    if tramitacao_encerrada == 'NAO': 
                        aprovado = 'TRAMITACAO'
                    elif 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jur' in listagem['Situacao'][i]:
                        aprovado = 'SIM'
                    elif 'Retirad' in listagem['Situacao'][i] or 'Rejeitad' in listagem['Situacao'][i] or 'Vetad' in listagem['Situacao'][i]:# era 'Vetado Totalmente'\
                        aprovado = 'NAO'
                    #elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                    #    if listagem['Situacao'][i] == 'Arquivada': 
                    #        aprovado = 'NAO'        
                    #    else:        
                    #        aprovado = 'SIM'
                    #elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                    else:
                        try:
                            # Limpeza para pegar o id do projeto dentro da prefeitura
                            #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                            if(re.search('APROVAD', retorno)):
                                aprovado = 'SIM'
                            else:
                                aprovado = 'NAO'
                        except:
                            aprovado = 'Informacao nao encontrada'
                            notfound_count_aprv += 1

                except:
                    failed = True
                    time.sleep(180)

                listagem['Data_Projeto'][i] = data_projeto
                listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
                listagem['Aprovado'][i] = aprovado

                #resposta.close()
                session.close()
                time.sleep(0.5)

        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req or req_count > max_req:
            break
            
    if failed_req > max_failed_req or req_count > max_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' and listagem['Tramitacao_Encerrada'][i] == '' and listagem['Aprovado'][i] == '' and listagem['Id_Projeto_Camara'][i] != '':
            count +=1

now = datetime.now()
dthr_finalizado = now.strftime("%d-%m-%Y_%H-%M-%S")

try:
    # Exportacao da listagem no estado atual para arquivo csv-
    file = 'ListagemExport2016-2022_checkpoint_' + dthr_finalizado
    path_to_file = '../ListagemExp/'+file+'.csv'
    path = Path(path_to_file)

    exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
                    'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

    if path.is_file():
        exp_header = False
    else:    
        exp_header = True

    listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

    export = True

except:
    export = False

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
elif req_count > max_req:
    print(f'Processo finalizado em {req_count} requisicoes. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
else:
    print(f'Fim do processo. Nao encontrados: data = {notfound_count_dt}, tramitacao = {notfound_count_tram}, aprovador = {notfound_count_aprv}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')

O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados: data = 11, tramitacao = 0, aprovador = 0. Finalizado em: 14-11-2022_21-20-12. Status da exportacao: True.


### Exportacao da listagem no estado atual 

In [11]:
# Exportacao da listagem no estado atual para arquivo csv
now =datetime.now()

file = 'ListagemExport2012-2015_trab_altmanual' + now.strftime("%d-%m-%Y_%H-%M-%S")
path_to_file = '../ListagemExp/Trabalhados/'+file+'.csv'
path = Path(path_to_file)

exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
               'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 
print(path)

..\ListagemExp\Trabalhados\ListagemExport2012-2015_trab_altmanual09-11-2022_14-58-07.csv


### Identificacao do estado da importacao dos dados

In [15]:
tem=0
naovaiter=0
naotem=0

#oid = ''
for i in listagem.index:
    if listagem['Data_Projeto'][i] != '' and listagem['Tramitacao_Encerrada'][i] != '' and listagem['Aprovado'][i] != '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
        tem+=1
    elif listagem['Codigo_Tipo'][i] in ('DESCONHECIDO','EMENDA'):
        naovaiter+=1
    else:
        naotem+=1
        #oid=i
print(tem)
print(naotem)
print(naovaiter)
#print(oid)   

10019
51705
424
